In [1]:
import pandas as pd
import sklearn
import numpy as np

In [28]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [13]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,auc
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

In [3]:
df = pd.read_csv('criminal_train.csv')

In [7]:
df.columns

Index(['PERID', 'IFATHER', 'NRCH17_2', 'IRHHSIZ2', 'IIHHSIZ2', 'IRKI17_2',
       'IIKI17_2', 'IRHH65_2', 'IIHH65_2', 'PRXRETRY', 'PRXYDATA', 'MEDICARE',
       'CAIDCHIP', 'CHAMPUS', 'PRVHLTIN', 'GRPHLTIN', 'HLTINNOS', 'HLCNOTYR',
       'HLCNOTMO', 'HLCLAST', 'HLLOSRSN', 'HLNVCOST', 'HLNVOFFR', 'HLNVREF',
       'HLNVNEED', 'HLNVSOR', 'IRMCDCHP', 'IIMCDCHP', 'IRMEDICR', 'IIMEDICR',
       'IRCHMPUS', 'IICHMPUS', 'IRPRVHLT', 'IIPRVHLT', 'IROTHHLT', 'IIOTHHLT',
       'HLCALLFG', 'HLCALL99', 'ANYHLTI2', 'IRINSUR4', 'IIINSUR4', 'OTHINS',
       'CELLNOTCL', 'CELLWRKNG', 'IRFAMSOC', 'IIFAMSOC', 'IRFAMSSI',
       'IIFAMSSI', 'IRFSTAMP', 'IIFSTAMP', 'IRFAMPMT', 'IIFAMPMT', 'IRFAMSVC',
       'IIFAMSVC', 'IRWELMOS', 'IIWELMOS', 'IRPINC3', 'IRFAMIN3', 'IIPINC3',
       'IIFAMIN3', 'GOVTPROG', 'POVERTY3', 'TOOLONG', 'TROUBUND', 'PDEN10',
       'COUTYP2', 'MAIIN102', 'AIIND102', 'ANALWT_C', 'VESTR', 'VEREP',
       'Criminal'],
      dtype='object')

In [14]:
df = df.dropna()

In [15]:
cols = ['IIKI17_2', 'IIKI17_2','IIHH65_2','IIMCDCHP','IIMEDICR','IICHMPUS','IIPRVHLT','IIOTHHLT','IIINSUR4',
              'IIFAMSOC','IIFAMSSI','IIFSTAMP','IIFAMPMT','IIFAMSVC','IIFAMSVC','IIFAMIN3']

In [16]:
df_col = df[cols]
df_col.describe()

In [23]:
df2 = df.drop(['PERID','IIKI17_2', 'IIKI17_2','IIHH65_2','IIMCDCHP','IIMEDICR','IICHMPUS','IIPRVHLT','IIOTHHLT','IIINSUR4',
              'IIFAMSOC','IIFAMSSI','IIFSTAMP','IIFAMPMT','IIFAMSVC','IIFAMSVC','IIFAMIN3'], axis=1, inplace=False)

In [25]:
pd.Categorical(df2)

[IFATHER, NRCH17_2, IRHHSIZ2, IIHHSIZ2, IRKI17_2, ..., AIIND102, ANALWT_C, VESTR, VEREP, Criminal]
Length: 57
Categories (57, object): [AIIND102, ANALWT_C, ANYHLTI2, CAIDCHIP, ..., TOOLONG, TROUBUND, VEREP, VESTR]

In [88]:
df['Criminal'].value_counts()

0    42543
1     3175
Name: Criminal, dtype: int64

In [26]:
X = df2.drop(['Criminal'], axis=1)
y = df2["Criminal"]

In [78]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = .33, random_state = 1)

In [79]:
x_train = xgb.DMatrix(x_train, label=y_train)
x_test = xgb.DMatrix(x_test)

In [89]:
param = {'max_depth':3, 'eta':0.1, 'silent':1,'gamma' : 4,'max_delta_step': 8,
         'scale_pos_weight' : 0.07,
         'objective':'binary:logistic','min_child_weight' : 1}
num_round = 1000
bst = xgb.train(param, x_train, num_round)

In [86]:
?xgb.train

In [90]:
y_pred = bst.predict(x_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.01%


In [31]:
model = XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=1000, 
                      silent=True, objective='binary:logistic', booster='gbtree',
                      n_jobs=3, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, 
                      subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0,
                      reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0)
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [33]:
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 95.15%


In [63]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(200, activation='relu', input_shape = (56,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))


In [68]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy'  ,metrics=['accuracy'])
history = model.fit(np.array(x_train), np.array(y_train), nb_epoch=20, batch_size=5, validation_data=( np.array(x_test),
                                                                                                      np.array(y_test)))

C:\Users\hp\Anaconda3\envs\tf\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 30631 samples, validate on 15087 samples
Epoch 1/20
30631/30631 [==============================] - 6s - loss: 14.8390 - acc: 0.0692 - val_loss: 14.8276 - val_acc: 0.0699
Epoch 2/20
14225/30631 [============>.................] - ETA: 2s - loss: 14.8631 - acc: 0.0677

KeyboardInterrupt: 